In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "0aStgWSO0E0E"
      },
      "source": [
        "# Classification"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "1eLEkw5O0ECa"
      },
      "source": [
        "## Objectives\n",
        "\n",
        "*   Fit and evaluate a classification model to predict if a prospect will churn or not.\n",
        "\n",
        "\n",
        "## Inputs\n",
        "\n",
        "* outputs/datasets/collection/TelcoCustomerChurn.csv\n",
        "* Instructions on which variables to use for data cleaning and feature engineering. They are found in each respective notebook.\n",
        "\n",
        "## Outputs\n",
        "\n",
        "* Train set (features and target)\n",
        "* Test set (features and target)\n",
        "* Data cleaning and Feature Engineering pipeline\n",
        "* Modeling pipeline\n",
        "* Feature importance plot\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "9uWZXH9LwoQg"
      },
      "source": [
        "---"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Change working directory"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "We need to change the working directory from its current folder to its parent folder\n",
        "* We access the current directory with os.getcwd()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import os\n",
        "current_dir = os.getcwd()\n",
        "current_dir"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "We want to make the parent of the current directory the new current directory.\n",
        "* os.path.dirname() gets the parent directory\n",
        "* os.chir() defines the new current directory"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "os.chdir(os.path.dirname(current_dir))\n",
        "print(\"You set a new current directory\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "Confirm the new current directory"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "current_dir = os.getcwd()\n",
        "current_dir"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "OSpFreVRiuM3"
      },
      "source": [
        "---"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "-mavJ8DibrcQ"
      },
      "source": [
        "# Step 1: Load Data"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "Xk7DU_ekbtX8"
      },
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "import pandas as pd\n",
        "df = (pd.read_csv(\"outputs/datasets/collection/TelcoCustomerChurn.csv\")\n",
        "      .drop(labels=['tenure', 'customerID', 'TotalCharges'], axis=1)  \n",
        "                    # target variable for regressor, remove from classifier  \n",
        "                    # drop other variables we will not need for this project\n",
        "  )\n",
        "\n",
        "print(df.shape)\n",
        "df.head(3)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Ofil7xTpm6l9"
      },
      "source": [
        "---"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "krjAk78Tbyhv"
      },
      "source": [
        "# Step 2: ML Pipeline with all data"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "FfCsXhBYVBJw"
      },
      "source": [
        "## ML pipeline for Data Cleaning and Feature Engineering"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "C6keis6ao8LA"
      },
      "outputs": [],
      "source": [
        "from sklearn.pipeline import Pipeline\n",
        "\n",
        "# Feature Engineering\n",
        "from feature_engine.selection import SmartCorrelatedSelection\n",
        "from feature_engine.encoding import OrdinalEncoder\n",
        "\n",
        "\n",
        "def PipelineDataCleaningAndFeatureEngineering():\n",
        "    pipeline_base = Pipeline([\n",
        "        (\"OrdinalCategoricalEncoder\", OrdinalEncoder(encoding_method='arbitrary',\n",
        "                                                     variables=['gender', 'Partner', 'Dependents', 'PhoneService',\n",
        "                                                                'MultipleLines', 'InternetService', 'OnlineSecurity',\n",
        "                                                                'OnlineBackup', 'DeviceProtection', 'TechSupport',\n",
        "                                                                'StreamingTV', 'StreamingMovies', 'Contract',\n",
        "                                                                'PaperlessBilling', 'PaymentMethod'])),\n",
        "\n",
        "        (\"SmartCorrelatedSelection\", SmartCorrelatedSelection(variables=None,\n",
        "         method=\"spearman\", threshold=0.6, selection_method=\"variance\")),\n",
        "\n",
        "    ])\n",
        "\n",
        "    return pipeline_base\n",
        "\n",
        "\n",
        "PipelineDataCleaningAndFeatureEngineering()\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "H_7BXNYMULrf"
      },
      "source": [
        "## ML Pipeline for Modelling and Hyperparameter Optimisation"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "PYR4hz6-Ldvo"
      },
      "outputs": [],
      "source": [
        "# Feat Scaling\n",
        "from sklearn.preprocessing import StandardScaler\n",
        "\n",
        "# Feat Selection\n",
        "from sklearn.feature_selection import SelectFromModel\n",
        "\n",
        "# ML algorithms\n",
        "from sklearn.linear_model import LogisticRegression\n",
        "from sklearn.tree import DecisionTreeClassifier\n",
        "from sklearn.ensemble import RandomForestClassifier\n",
        "from sklearn.ensemble import GradientBoostingClassifier\n",
        "from sklearn.ensemble import ExtraTreesClassifier\n",
        "from sklearn.ensemble import AdaBoostClassifier\n",
        "from xgboost import XGBClassifier\n",
        "\n",
        "\n",
        "def PipelineClf(model):\n",
        "    pipeline_base = Pipeline([\n",
        "        (\"scaler\", StandardScaler()),\n",
        "        (\"feat_selection\", SelectFromModel(model)),\n",
        "        (\"model\", model),\n",
        "    ])\n",
        "\n",
        "    return pipeline_base\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "KM_hrtfjLj85"
      },
      "source": [
        "Custom Class for Hyperparameter Optimisation"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "NpTcVDtQ5RMc"
      },
      "outputs": [],
      "source": [
        "from sklearn.model_selection import GridSearchCV\n",
        "\n",
        "\n",
        "class HyperparameterOptimizationSearch:\n",
        "\n",
        "    def __init__(self, models, params):\n",
        "        self.models = models\n",
        "        self.params = params\n",
        "        self.keys = models.keys()\n",
        "        self.grid_searches = {}\n",
        "\n",
        "    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):\n",
        "        for key in self.keys:\n",
        "            print(f\"\\nRunning GridSearchCV for {key} \\n\")\n",
        "\n",
        "            model = PipelineClf(self.models[key])\n",
        "            params = self.params[key]\n",
        "            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,\n",
        "                              verbose=verbose, scoring=scoring, )\n",
        "            gs.fit(X, y)\n",
        "            self.grid_searches[key] = gs\n",
        "\n",
        "    def score_summary(self, sort_by='mean_score'):\n",
        "        def row(key, scores, params):\n",
        "            d = {\n",
        "                'estimator': key,\n",
        "                'min_score': min(scores),\n",
        "                'max_score': max(scores),\n",
        "                'mean_score': np.mean(scores),\n",
        "                'std_score': np.std(scores),\n",
        "            }\n",
        "            return pd.Series({**params, **d})\n",
        "\n",
        "        rows = []\n",
        "        for k in self.grid_searches:\n",
        "            params = self.grid_searches[k].cv_results_['params']\n",
        "            scores = []\n",
        "            for i in range(self.grid_searches[k].cv):\n",
        "                key = \"split{}_test_score\".format(i)\n",
        "                r = self.grid_searches[k].cv_results_[key]\n",
        "                scores.append(r.reshape(len(params), 1))\n",
        "\n",
        "            all_scores = np.hstack(scores)\n",
        "            for p, s in zip(params, all_scores):\n",
        "                rows.append((row(k, s, p)))\n",
        "\n",
        "        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)\n",
        "        columns = ['estimator', 'min_score',\n",
        "                   'mean_score', 'max_score', 'std_score']\n",
        "        columns = columns + [c for c in df.columns if c not in columns]\n",
        "        return df[columns], self.grid_searches\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "eUcOp83jy0QG"
      },
      "source": [
        "## Split Train and Test Set"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "0vqzNI2zF1sZ"
      },
      "outputs": [],
      "source": [
        "from sklearn.model_selection import train_test_split\n",
        "X_train, X_test, y_train, y_test = train_test_split(\n",
        "    df.drop(['Churn'], axis=1),\n",
        "    df['Churn'],\n",
        "    test_size=0.2,\n",
        "    random_state=0,\n",
        ")\n",
        "\n",
        "print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "4zBysp0tyqR2"
      },
      "source": [
        "## Handle Target Imbalance"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "MsQRvnn1GI_d"
      },
      "outputs": [],
      "source": [
        "pipeline_data_cleaning_feat_eng = PipelineDataCleaningAndFeatureEngineering()\n",
        "X_train = pipeline_data_cleaning_feat_eng.fit_transform(X_train)\n",
        "X_test = pipeline_data_cleaning_feat_eng.transform(X_test)\n",
        "print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "wuq3902arZAz"
      },
      "source": [
        "Check Train Set Target distribution"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "I28ACrp-rPgF"
      },
      "outputs": [],
      "source": [
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "sns.set_style(\"whitegrid\")\n",
        "y_train.value_counts().plot(kind='bar', title='Train Set Target Distribution')\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "-OgoR6lTrKqY"
      },
      "source": [
        "Use SMOTE (Synthetic Minority Oversampling TEchnique) to balance Train Set target"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "tP1JIwXNEsXO"
      },
      "outputs": [],
      "source": [
        "from imblearn.over_sampling import SMOTE\n",
        "oversample = SMOTE(sampling_strategy='minority', random_state=0)\n",
        "X_train, y_train = oversample.fit_resample(X_train, y_train)\n",
        "print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "vTJO6V5zrdnw"
      },
      "source": [
        "Check Train Set Target distribution after resampling"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "iQdvEvNRG80Y"
      },
      "outputs": [],
      "source": [
        "import matplotlib.pyplot as plt\n",
        "y_train.value_counts().plot(kind='bar', title='Train Set Target Distribution')\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "j2xTTXMayvo6"
      },
      "source": [
        "## Grid Search CV - Sklearn"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "fizLJ_YQ6elb"
      },
      "source": [
        "### Use standard hyperparameters to find most suitable algorithm "
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "kMgswohfKBda"
      },
      "outputs": [],
      "source": [
        "models_quick_search = {\n",
        "    \"LogisticRegression\": LogisticRegression(random_state=0),\n",
        "    \"XGBClassifier\": XGBClassifier(random_state=0),\n",
        "    \"DecisionTreeClassifier\": DecisionTreeClassifier(random_state=0),\n",
        "    \"RandomForestClassifier\": RandomForestClassifier(random_state=0),\n",
        "    \"GradientBoostingClassifier\": GradientBoostingClassifier(random_state=0),\n",
        "    \"ExtraTreesClassifier\": ExtraTreesClassifier(random_state=0),\n",
        "    \"AdaBoostClassifier\": AdaBoostClassifier(random_state=0),\n",
        "}\n",
        "\n",
        "params_quick_search = {\n",
        "    \"LogisticRegression\": {},\n",
        "    \"XGBClassifier\": {},\n",
        "    \"DecisionTreeClassifier\": {},\n",
        "    \"RandomForestClassifier\": {},\n",
        "    \"GradientBoostingClassifier\": {},\n",
        "    \"ExtraTreesClassifier\": {},\n",
        "    \"AdaBoostClassifier\": {},\n",
        "}\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "GXu0Ryeown7N"
      },
      "source": [
        "Quick GridSearch CV - Binary Classifier"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "O7eLJcKEKBlQ"
      },
      "outputs": [],
      "source": [
        "from sklearn.metrics import make_scorer, recall_score\n",
        "search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)\n",
        "search.fit(X_train, y_train,\n",
        "           scoring =  make_scorer(recall_score, pos_label=1),\n",
        "           n_jobs=-1, cv=5)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "g0bkL-IxwnJx"
      },
      "source": [
        "Check results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "YpFOc7OAKMuz"
      },
      "outputs": [],
      "source": [
        "grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')\n",
        "grid_search_summary "
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ewezVDt46jTJ"
      },
      "source": [
        "### Do an extensive search on the most suitable algorithm to find the best hyperparameter configuration."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Z1WozH5frBQ9"
      },
      "source": [
        "Define model and parameters, for Extensive Search"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "sDT_WMUErBRB"
      },
      "outputs": [],
      "source": [
        "models_search = {\n",
        "    \"XGBClassifier\":XGBClassifier(random_state=0),\n",
        "}\n",
        "\n",
        "# documentation to help on hyperparameter list: \n",
        "# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn\n",
        "\n",
        "# We will not conduct an extensive search, since the focus\n",
        "# is on how to combine all knowledge in an applied project.\n",
        "# In a workplace project, you may spend more time in this step\n",
        "params_search = {\n",
        "    \"XGBClassifier\":{\n",
        "        'model__learning_rate': [1e-1,1e-2,1e-3], \n",
        "        'model__max_depth': [3,10,None],\n",
        "    }\n",
        "}"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "BP2Ua0FGrBRC"
      },
      "source": [
        "Extensive GridSearch CV - Binary Classifier"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "WK1s893orBRD"
      },
      "outputs": [],
      "source": [
        "from sklearn.metrics import recall_score, make_scorer\n",
        "search = HyperparameterOptimizationSearch(models=models_search, params=params_search)\n",
        "search.fit(X_train, y_train,\n",
        "           scoring =  make_scorer(recall_score, pos_label=1),\n",
        "           n_jobs=-1, cv=5)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "l8oVKtHyr-X8"
      },
      "source": [
        "Check results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "8AFyZ6-pr9tN"
      },
      "outputs": [],
      "source": [
        "grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')\n",
        "grid_search_summary "
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "Get best model name programmatically"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "best_model = grid_search_summary.iloc[0,0]\n",
        "best_model"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "htAXEVFpwiBV"
      },
      "source": [
        "Parameters for best model"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "oDIt27RdKOG8"
      },
      "outputs": [],
      "source": [
        "best_parameters = grid_search_pipelines[best_model].best_params_\n",
        "best_parameters"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "eAnJQlDlw1FE"
      },
      "source": [
        "Define the best clf pipeline"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "zLotNfy4MKDE"
      },
      "outputs": [],
      "source": [
        "pipeline_clf = grid_search_pipelines[best_model].best_estimator_\n",
        "pipeline_clf"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "UgdxKijH6qJS"
      },
      "source": [
        "## Assess feature importance"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "X_train.head(3)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "n30pl2dowzW3"
      },
      "source": [
        "* With the current model, we can assess with `.features_importances_`"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "4XGczhv2uo2C"
      },
      "outputs": [],
      "source": [
        "# create DataFrame to display feature importance\n",
        "df_feature_importance = (pd.DataFrame(data={\n",
        "    'Feature': X_train.columns[pipeline_clf['feat_selection'].get_support()],\n",
        "    'Importance': pipeline_clf['model'].feature_importances_})\n",
        "    .sort_values(by='Importance', ascending=False)\n",
        ")\n",
        "\n",
        "# re-assign best_features order\n",
        "best_features = df_feature_importance['Feature'].to_list()\n",
        "\n",
        "# Most important features statement and plot\n",
        "print(f\"* These are the {len(best_features)} most important features in descending order. \"\n",
        "      f\"The model was trained on them: \\n{df_feature_importance['Feature'].to_list()}\")\n",
        "\n",
        "df_feature_importance.plot(kind='bar', x='Feature', y='Importance')\n",
        "plt.show()\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "hXtmFP_Ulpnd"
      },
      "source": [
        "## Evaluate Pipeline on Train and Test Sets"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "myG6tDSGan4r"
      },
      "outputs": [],
      "source": [
        "from sklearn.metrics import classification_report, confusion_matrix\n",
        "\n",
        "\n",
        "def confusion_matrix_and_report(X, y, pipeline, label_map):\n",
        "\n",
        "    prediction = pipeline.predict(X)\n",
        "\n",
        "    print('---  Confusion Matrix  ---')\n",
        "    print(pd.DataFrame(confusion_matrix(y_true=prediction, y_pred=y),\n",
        "          columns=[[\"Actual \" + sub for sub in label_map]],\n",
        "          index=[[\"Prediction \" + sub for sub in label_map]]\n",
        "          ))\n",
        "    print(\"\\n\")\n",
        "\n",
        "    print('---  Classification Report  ---')\n",
        "    print(classification_report(y, prediction, target_names=label_map), \"\\n\")\n",
        "\n",
        "\n",
        "def clf_performance(X_train, y_train, X_test, y_test, pipeline, label_map):\n",
        "    print(\"#### Train Set #### \\n\")\n",
        "    confusion_matrix_and_report(X_train, y_train, pipeline, label_map)\n",
        "\n",
        "    print(\"#### Test Set ####\\n\")\n",
        "    confusion_matrix_and_report(X_test, y_test, pipeline, label_map)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "qpUfEAGlW5aK"
      },
      "source": [
        "Evaluation: We cross check with metrics defined at ML business case\n",
        "* 80% Recall for Churn, on train and test set\n",
        "* 80% Precision for no Churn on train and test set. "
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "umWjIvGMNLig"
      },
      "outputs": [],
      "source": [
        "clf_performance(X_train=X_train, y_train=y_train,\n",
        "                X_test=X_test, y_test=y_test,\n",
        "                pipeline=pipeline_clf,\n",
        "                label_map= ['No Churn', 'Churn'] \n",
        "                )"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "7WgttWjtHHOQ"
      },
      "source": [
        "# Step 3: Refit pipeline with best features"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "kCyOyebVHVmA"
      },
      "source": [
        "## Refit ML Pipeline and Resampling"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "R4PpI2sKC5IL"
      },
      "source": [
        "In theory, a pipeline fitted **using only the most important features** should give the same result as the one fitted with **all variables and feature selection**\n",
        "\n",
        "* However, in this project we have a step for feature augmentation, which is to balance the target Train Set using SMOTE().\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Km_-hW0f68DP"
      },
      "source": [
        "## Rewrite ML pipeline for Data Cleaning and Feature Engineering"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "best_features"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "EBeckIjkCa4k"
      },
      "source": [
        "New Pipeline for DataCleaning And FeatureEngineering"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "bc8ptvFiHJmb"
      },
      "outputs": [],
      "source": [
        "def PipelineDataCleaningAndFeatureEngineering():\n",
        "    pipeline_base = Pipeline([\n",
        "\n",
        "        (\"OrdinalCategoricalEncoder\", OrdinalEncoder(encoding_method='arbitrary',\n",
        "                                                     variables=['InternetService', 'Contract'])),\n",
        "\n",
        "\n",
        "        # we don't need SmartCorrelatedSelection\n",
        "    ])\n",
        "\n",
        "    return pipeline_base\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "uGNs9PU16_Ls"
      },
      "source": [
        "## Rewrite ML Pipeline for Modelling"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "gpjmxzTbCXlg"
      },
      "source": [
        "Function for Pipeline optmisation"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "8E76QmoMEWA0"
      },
      "outputs": [],
      "source": [
        "# Pipeline Optmization: Model\n",
        "def PipelineClf(model):\n",
        "    pipeline_base = Pipeline([\n",
        "        (\"scaler\", StandardScaler()),\n",
        "        # no feature selection needed anymore!!! We know which features to use already!\n",
        "        (\"model\", model),\n",
        "    ])\n",
        "\n",
        "    return pipeline_base\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "75hfh3o5GhoU"
      },
      "source": [
        "## Split Train Test Set, considering only with best features"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "x6dX0VeKGhod"
      },
      "outputs": [],
      "source": [
        "from sklearn.model_selection import train_test_split\n",
        "X_train, X_test, y_train, y_test = train_test_split(\n",
        "    df.drop(['Churn'], axis=1),\n",
        "    df['Churn'],\n",
        "    test_size=0.2,\n",
        "    random_state=0,\n",
        ")\n",
        "\n",
        "print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "c19a3t6jI6H6"
      },
      "source": [
        "We filter only the most important variables"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "p5Acb9T_GXjU"
      },
      "outputs": [],
      "source": [
        "X_train = X_train.filter(best_features)\n",
        "X_test = X_test.filter(best_features)\n",
        "\n",
        "print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)\n",
        "X_train.head(3)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "sjOcRGheGhof"
      },
      "source": [
        "## Handle Target Imbalance"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "KbQda_pcGhof"
      },
      "outputs": [],
      "source": [
        "pipeline_data_cleaning_feat_eng = PipelineDataCleaningAndFeatureEngineering()\n",
        "X_train = pipeline_data_cleaning_feat_eng.fit_transform(X_train)\n",
        "X_test = pipeline_data_cleaning_feat_eng.transform(X_test)\n",
        "print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "EQxIFw3KGhog"
      },
      "source": [
        "Check Train Set Target distribution"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "7ZQyth-2Ghog"
      },
      "outputs": [],
      "source": [
        "import matplotlib.pyplot as plt\n",
        "y_train.value_counts().plot(kind='bar', title='Train Set Target Distribution')\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "N9FbCbIrGhoh"
      },
      "source": [
        "Use SMOTE to balance Train Set target"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "OtbWft5VGhoh"
      },
      "outputs": [],
      "source": [
        "from imblearn.over_sampling import SMOTE\n",
        "oversample = SMOTE(sampling_strategy='minority', random_state=0)\n",
        "X_train, y_train = oversample.fit_resample(X_train, y_train)\n",
        "print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "YozwzI9eGhoh"
      },
      "source": [
        "Check Train Set Target distribution after SMOTE"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "TyL99cYMGhoi"
      },
      "outputs": [],
      "source": [
        "y_train.value_counts().plot(kind='bar',title='Train Set Target Distribution')\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "b_WjvD_QIJ_L"
      },
      "source": [
        "## Grid Search CV: Sklearn"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ESkqrySI7N6u"
      },
      "source": [
        "Using the most suitable model from the last section and its best hyperparameter configuration."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "F6fFaXDOIJ_M"
      },
      "source": [
        "We are using the same model from  the last GridCV search"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "H7F0z__h1qSA"
      },
      "outputs": [],
      "source": [
        "models_search   # XGBClassifier"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "qRteBPgd3ldU"
      },
      "source": [
        "And the best parameters from the last GridCV search "
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "IbGNBeZk3V8r"
      },
      "outputs": [],
      "source": [
        "best_parameters"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "YlLJP5Ds3rYp"
      },
      "source": [
        "You will need to type in manually since the hyperparameter values have to be a list. The previous dictionary is not in this format."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "9bC8RmE-2Mi2"
      },
      "outputs": [],
      "source": [
        "params_search = {'XGBClassifier':  {\n",
        "    'model__learning_rate': [0.01],   # the value should be in []\n",
        "    'model__max_depth': [3]},  # the value should be in []\n",
        "}\n",
        "params_search\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "GrRJNywsIJ_M"
      },
      "source": [
        "GridSearch CV"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "yv5nO6cJP9fX"
      },
      "outputs": [],
      "source": [
        "from sklearn.metrics import recall_score, make_scorer\n",
        "quick_search = HyperparameterOptimizationSearch(\n",
        "    models=models_search, params=params_search)\n",
        "quick_search.fit(X_train, y_train,\n",
        "                 scoring=make_scorer(recall_score, pos_label=1),\n",
        "                 n_jobs=-1, cv=5)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Yr_Yu9ykIJ_N"
      },
      "source": [
        "Check results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "fqIk1g95IJ_N"
      },
      "outputs": [],
      "source": [
        "grid_search_summary, grid_search_pipelines = quick_search.score_summary(sort_by='mean_score')\n",
        "grid_search_summary "
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "tZcP3yXpIJ_O"
      },
      "source": [
        "Define the best clf pipeline"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "P7Qe2jFEIJ_O"
      },
      "outputs": [],
      "source": [
        "best_model = grid_search_summary.iloc[0, 0]\n",
        "pipeline_clf = grid_search_pipelines[best_model].best_estimator_\n",
        "pipeline_clf"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "qXEXWvsb7Su0"
      },
      "source": [
        "## Assess feature importance"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "T8UGZ5bnIJ_P"
      },
      "outputs": [],
      "source": [
        "best_features = X_train.columns\n",
        "\n",
        "# create DataFrame to display feature importance\n",
        "df_feature_importance = (pd.DataFrame(data={\n",
        "    'Feature': best_features,\n",
        "    'Importance': pipeline_clf['model'].feature_importances_})\n",
        "    .sort_values(by='Importance', ascending=False)\n",
        ")\n",
        "\n",
        "\n",
        "# Most important features statement and plot\n",
        "print(f\"* These are the {len(best_features)} most important features in descending order. \"\n",
        "      f\"The model was trained on them: \\n{df_feature_importance['Feature'].to_list()}\")\n",
        "\n",
        "df_feature_importance.plot(kind='bar', x='Feature', y='Importance')\n",
        "plt.show()\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "nQF20xan7VuK"
      },
      "source": [
        "## Evaluate Pipeline on Train and Test Sets"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "Evaluation: We cross-check with metrics defined in the ML business case.\n",
        "* 80% Recall for Churn, on train and test set.\n",
        "* 80% Precision for no Churn on train and test set. "
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "1cpCj2lLHxB-"
      },
      "outputs": [],
      "source": [
        "clf_performance(X_train=X_train, y_train=y_train,\n",
        "                X_test=X_test, y_test=y_test,\n",
        "                pipeline=pipeline_clf,\n",
        "                label_map= ['No Churn', 'Churn'] \n",
        "                )"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "oBVunRgBqIXQ"
      },
      "source": [
        "# Step 4: Push files to Repo"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "yxnlKI5SJcoO"
      },
      "source": [
        "We will generate the following files\n",
        "* Train set\n",
        "* Test set\n",
        "* Data cleaning and Feature Engineering pipeline\n",
        "* Modeling pipeline\n",
        "* features importance plot"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "16bIOgs3J7OD"
      },
      "outputs": [],
      "source": [
        "import joblib\n",
        "import os\n",
        "\n",
        "version = 'v1'\n",
        "file_path = f'outputs/ml_pipeline/predict_churn/{version}'\n",
        "\n",
        "try:\n",
        "    os.makedirs(name=file_path)\n",
        "except Exception as e:\n",
        "    print(e)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "3e-gC6sa7hpj"
      },
      "source": [
        "## Train Set"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "hHZUZKJ5JiKn"
      },
      "source": [
        "* note that the variables **are transformed already** in X_train and the shape is 8266 - after SMOTE was applied."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "Sc4fzrdTJno1"
      },
      "outputs": [],
      "source": [
        "print(X_train.shape)\n",
        "X_train.head()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "Qzq7DgVTJnv3"
      },
      "outputs": [],
      "source": [
        "X_train.to_csv(f\"{file_path}/X_train.csv\", index=False)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "DzPsdNGX9gtf"
      },
      "outputs": [],
      "source": [
        "y_train"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "FMoT1cJ39g26"
      },
      "outputs": [],
      "source": [
        "y_train.to_csv(f\"{file_path}/y_train.csv\", index=False)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "OYatlgsj7pbB"
      },
      "source": [
        "## Test Set"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "tEKp3-dJJn3p"
      },
      "source": [
        "* note that the variables are transformed already in X_test"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "9UMg2vPtJqxM"
      },
      "outputs": [],
      "source": [
        "print(X_test.shape)\n",
        "X_test.head()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "uz2OqPW6Jqzv"
      },
      "outputs": [],
      "source": [
        "X_test.to_csv(f\"{file_path}/X_test.csv\", index=False)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "4pPTVz219xj3"
      },
      "outputs": [],
      "source": [
        "y_test"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "ap7fYYAm9xsj"
      },
      "outputs": [],
      "source": [
        "y_test.to_csv(f\"{file_path}/y_test.csv\", index=False)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "_ufHAplN7tdo"
      },
      "source": [
        "## ML Pipelines: Data Cleaning and Feat Eng pipeline and Modelling Pipeline"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "XAbbAO2r248W"
      },
      "source": [
        "We will save 2 pipelines: \n",
        "* Both should be used in conjunction to predict Live Data.\n",
        "* To predict on Train Set, Test Set we use only pipeline_clf, since the data is already processed.\n",
        "\n",
        "\n",
        "\n",
        "Pipeline responsible for Data Cleaning and Feature Engineering.\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "XCcAlvoG3CRm"
      },
      "outputs": [],
      "source": [
        "pipeline_data_cleaning_feat_eng"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "AaHdCf4HKBLg"
      },
      "outputs": [],
      "source": [
        "joblib.dump(value=pipeline_data_cleaning_feat_eng ,\n",
        "            filename=f\"{file_path}/clf_pipeline_data_cleaning_feat_eng.pkl\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "XE-iU6TL3LVI"
      },
      "source": [
        "* Pipeline responsible for Feature Scaling, and Model"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "_zEBxfvBqI29"
      },
      "outputs": [],
      "source": [
        "pipeline_clf"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "ObL5Iz8tKdsZ"
      },
      "outputs": [],
      "source": [
        "joblib.dump(value=pipeline_clf ,\n",
        "            filename=f\"{file_path}/clf_pipeline_model.pkl\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "yqEUyLG27v9N"
      },
      "source": [
        "## Feature Importance plot"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "wBiqB55L1Qhk"
      },
      "outputs": [],
      "source": [
        "df_feature_importance.plot(kind='bar',x='Feature',y='Importance')\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "NR0taWpn1RuD"
      },
      "outputs": [],
      "source": [
        "df_feature_importance.plot(kind='bar', x='Feature', y='Importance')\n",
        "plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "Good job, you should clear outputs, then run git commands to push files to the repo. Next, move on to Predict Tenure notebook"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "---"
      ]
    }
  ],
  "metadata": {
    "accelerator": "GPU",
    "colab": {
      "name": "Modeling and Evaluation - Predict Customer Churn.ipynb",
      "provenance": []
    },
    "interpreter": {
      "hash": "8b8334dab9339717f727a1deaf837b322d7a41c20d15cc86be99a8e69ceec8ce"
    },
    "kernelspec": {
      "display_name": "Python 3.8.12 64-bit ('3.8.12': pyenv)",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.8.12 (default, Sep 27 2022, 15:56:02) \n[GCC 9.4.0]"
    },
    "orig_nbformat": 2
  },
  "nbformat": 4,
  "nbformat_minor": 2
}